In [3]:
import tensorflow as tf
import numpy as np

In [7]:
# create a tf constant
const = tf.constant(2.0, name = "constant")

#create the tf variables
# b = tf.Variable(2.0, name = "b")
# -----------------------------------------------------------------------------------------------
# create TensorFlow variables as Placeholder if we dont knwo what exact values b will have
# The first argument is the datatype, shape of the Input, name.
# We use none because our input vector has the size of ?x1 (Unknown rows, one column)
b = tf.placeholder(tf.float32,[None, 1], name = 'b')
# later when we run the code, we must "feed-in" the real data. This must be a python dictionary!
# ------------------------------------------------------------------------------------------------
c = tf.Variable(1.0, name = "c")

#creation of operations
d = tf.add(b, c, name = "d")
e = tf.add(c, const, name = "e")
a = tf.multiply(d, e, name = "a")

# set-up the variable initialization
init_op = tf.global_variables_initializer()

# start the session
with tf.Session() as sess:
    # initialize the variables
    sess.run(init_op)
    # compute the output
    # a_out = sess.run(a) Standard Case
    a_out = sess.run(a, feed_dict = {b: np.arange(0,10)[:, np.newaxis]})
print(a_out)
    

[[ 3.]
 [ 6.]
 [ 9.]
 [12.]
 [15.]
 [18.]
 [21.]
 [24.]
 [27.]
 [30.]]


In [6]:
x = np.arange(0,10)
print(x)
y = np.arange(0, 10)[:, np.newaxis]
print(y)

[0 1 2 3 4 5 6 7 8 9]
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]


# A Neural Network Example
Our goal is to build a simple 3-layer ANN.
Furthermore, we are using the MNIST Dataset (28x28 pixel images of handwritten digits from 0 - 9).
It has 70k images, from which we use 55K for Training 10k for Testing and 5k for validation


In [12]:
# loading the data (TF has its own loader for the MNIST dataset)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
# one_hot means that instead of having a specific label on every row, we want to have e.g. for the number
# 4 the label 1 and every row which is not a 4 gets the label 0.

#---------------------------------------------------------------------------------

# Now we can set up the placeholder and variables for the training
# Hyperparameters:
learning_rate = 0.5
epochs = 10
batch_size = 100

# delare the trianing data placeholders
x = tf.placeholder(tf.float32, [None, 784]) # 28 x 28 = 784
# delare the output data placeholder
y = tf.placeholder(tf.float32, [None, 10]) # 10 nodes corresponding to 10 possible digits

# ------------------------------------------------------------------------------------

# Now we need to set-up the weight and bias variables
# Connecting Input_Layer and Hidden_Layer (The Hidden_Layer will have 300 Nodes)
Wi_h = tf.Variable(tf.random_normal([784, 300], stddev = 0.03), name = "Wi_h")
bi_h = tf.Variable(tf.random_normal([300], stddev = 0.03), name = "bi_h")

# Connecting Hidden_Layer and Output_Layer
Wh_o = tf.Variable(tf.random_normal([300, 10], stddev = 0.03), name = "Wo_h")
bh_o = tf.Variable(tf.random_normal([10], stddev = 0.03), name = "bh_o")

# We use a normal distribution with the mean = 0 and a standard deviation = 0.03 to initialize the weights
# and biases randomly. Therefore, the weights and biases will be randomly small between 0 and 0.03

# -----------------------------------------------------------------------------------------

# Now we setup the input-nodes and the activation function for the hidden layer neurons
# Remember: each neuron does perform a dot-product of its inputs (784 greyvalues) with its associated weights
# Remember: the result of this muliplication will them be "poured" into an acitvation function to decide 
# Remember: if the neuron will fire or not. 

# calculate the Output for the Hidden layer
dot_product_plus_bias = tf.add(tf.matmul(x, Wi_h), bi_h) # matmul = matrix multiplication
hidden_out = tf.nn.relu(dot_product_plus_bias) # we use the relu function as the activiation function

# -------------------------------------------------------------------------------------------

# calculation of the output layer
dot_product_plus_bias_ho = tf.add(tf.matmul(hidden_out, Wh_o), bh_o)
y_ = tf.nn.softmax(dot_product_plus_bias_ho) # here we use the softmax function as an activiation function

# ---------------------------------------------------------------------------------------------

# calculation of the cost function for the backprop.
# in this case we use the cross entropy cost function. 
# since it uses log() we must clip the results of y_ between a super small value and 0.9999999
# to avoid a NaN and interuption of the training process

y_clipped = tf.clip_by_value(y_, 1e-10, 0.99999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1-y)*tf.log(1-y_clipped), axis = 1))

# ----------------------------------------------------------------------------------------------

# set up of the optimiser
# we are using the gradient descent algorithm to update the weights with the goal to minimize the cross entropy
# this is our backprop. 
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

# -----------------------------------------------------------------------------------------------

# initialize the variables
init_op = tf.global_variables_initializer()

# define the KPIs
correct_predicition = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# tf.equals returns True if the two arguments are equal
# tf.argmax returns the index of the maximum value in the matrix or vector
# Hence, correct_prediction will be a mx1 matrix with True ond False values
accuracy = tf.reduce_mean(tf.cast(correct_predicition, tf.float32))
# We want to calc the mean of the correct_prediction matrix
# first we must cast from boolean to float32 and then apply the tf.reduce_mean function

# -------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------

# Setting up the training
with tf.Session() as sess:
    # init the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels)/batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size = batch_size)
            _, c = sess.run([optimizer, cross_entropy],
                           feed_dict = {x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch", (epoch + 1), "cost = ", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch 1 cost =  0.488
Epoch 2 cost =  0.183
Epoch 3 cost =  0.126
Epoch 4 cost =  0.096
Epoch 5 cost =  0.076
Epoch 6 cost =  0.055
Epoch 7 cost =  0.042
Epoch 8 cost =  0.033
Epoch 9 cost =  0.021
Epoch 10 cost =  0.016
0.9798


In [11]:
print(c)

nan
